## Day 13: Day 13: Distress Signal

You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a distress signal.

Your handheld device must still not be working properly; the packets from the distress signal got decoded out of order. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify how many pairs of packets are in the right order.

For example:

```
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
```

Packet data consists of lists and integers. Each list starts with [, ends with ], and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called left and the second value is called right. Then:

- If **both values are integers**, the lower integer should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.

- If **both values are lists**, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.

- If **exactly one value is an integer**, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing [0,0,0] and 2, convert the right value to [2] (a list containing 2); the result is then found by instead comparing [0,0,0] and [2].

In [1]:
import pytest
from icecream import ic

In [2]:
from typing import Any
from itertools import zip_longest

def is_list(value: Any) -> bool:
    return isinstance(value, list)

assert is_list(23) is False
assert is_list([24]) is True

def compare(left_value, right_value, tron=False):
    if tron:
        ic(left_value, right_value)
    match is_list(left_value), is_list(right_value):    
        case (False, False): # Both are integer
            if tron:
                ic('Both are integers')
            if left_value < right_value:
                return True
            elif left_value > right_value:
                return False
            else:
                return None
        case (True, True): # Both are lists
            if tron:
                ic('Both are lists')
            for left, right in zip_longest(left_value, right_value, fillvalue=None):
                if left is None:
                    return True
                elif right is None:
                    return False
                result = compare(left, right, tron=tron)
                if result in {True, False}:
                    return result
        case (True, False):  # Left value is list, Right isn't
            if tron:
                ic('Left value is list, Right isn\'t')
            return compare(left_value, [right_value], tron=tron)
        case (False, True):  # First is not a list, but Right is
            if tron:
                ic('First is not a list, but Right is')
            return compare([left_value], right_value, tron=tron)
        
                
            
            
assert compare(3, 7) is True
assert compare(7, 3) is False
assert compare(5, 5) is None

assert compare([1,1,3,1,1], [1,1,5,1,1]) is True

Using these rules, you can determine which of the pairs in the example are in the right order:

```
== Pair 1 ==
- Compare [1,1,3,1,1] vs [1,1,5,1,1]
  - Compare 1 vs 1
  - Compare 1 vs 1
  - Compare 3 vs 5
    - Left side is smaller, so inputs are in the right order
```

In [3]:
assert compare([1,1,3,1,1], [1,1,5,1,1]) is True

```
== Pair 2 ==
- Compare [[1],[2,3,4]] vs [[1],4]
  - Compare [1] vs [1]
    - Compare 1 vs 1
  - Compare [2,3,4] vs 4
    - Mixed types; convert right to [4] and retry comparison
    - Compare [2,3,4] vs [4]
      - Compare 2 vs 4
        - Left side is smaller, so inputs are in the right order
```

In [4]:
assert compare([[1],[2,3,4]], [[1],4]) is True

```
== Pair 3 ==
- Compare [9] vs [[8,7,6]]
  - Compare 9 vs [8,7,6]
    - Mixed types; convert left to [9] and retry comparison
    - Compare [9] vs [8,7,6]
      - Compare 9 vs 8
        - Right side is smaller, so inputs are not in the right order
```

In [5]:
assert compare([9], [8, 7, 6]) is False

```
== Pair 4 ==
- Compare [[4,4],4,4] vs [[4,4],4,4,4]
  - Compare [4,4] vs [4,4]
    - Compare 4 vs 4
    - Compare 4 vs 4
  - Compare 4 vs 4
  - Compare 4 vs 4
  - Left side ran out of items, so inputs are in the right order
```

In [6]:
assert compare([[4,4],4,4], [[4,4],4,4,4]) is True

```
== Pair 5 ==
- Compare [7,7,7,7] vs [7,7,7]
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Right side ran out of items, so inputs are not in the right order
```

In [7]:
assert compare([7,7,7,7], [7,7,7]) is False

```
== Pair 6 ==
- Compare [] vs [3]
  - Left side ran out of items, so inputs are in the right order
```

In [8]:
assert compare([], [3]) is True

```
== Pair 7 ==
- Compare [[[]]] vs [[]]
  - Compare [[]] vs []
    - Right side ran out of items, so inputs are not in the right order
```

In [9]:
assert compare([[[]]], [[]]) is False

```
== Pair 8 ==
- Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
  - Compare 1 vs 1
  - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
    - Compare 2 vs 2
    - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
      - Compare 3 vs 3
      - Compare [4,[5,6,7]] vs [4,[5,6,0]]
        - Compare 4 vs 4
        - Compare [5,6,7] vs [5,6,0]
          - Compare 5 vs 5
          - Compare 6 vs 6
          - Compare 7 vs 0
            - Right side is smaller, so inputs are not in the right order
```

In [10]:
assert compare([1,[2,[3,[4,[5,6,7]]]],8,9], [1,[2,[3,[4,[5,6,0]]]],8,9]) is False

What are the indices of the pairs that are already in the right order? (The first pair has index 1, the second pair has index 2, and so on.) In the above example, the pairs in the right order are 1, 2, 4, and 6; the sum of these indices is **13**.

Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?

In [11]:
from ast import literal_eval

assert literal_eval("[1]") == [1]
assert literal_eval("[5,6,7]") == [5, 6, 7]
assert literal_eval("[4,[5,6,7]]") == [4, [5, 6, 7]]
assert literal_eval("[1,[2,[3,[4,[5,6,7]]]],8,9]") == [1, [2, [3, [4, [5, 6, 7]]]], 8, 9]

In [12]:
def batch(iterable, size=2):
    """Take an iterable and split it in several list
    of size _size_, except for the last one, which
    could have less elements.

    Example:

    >>> assert list(batch(range(1, 8), 3)) == [(1, 2, 3), (4, 5, 6), (7,)]
    """
    iterable = iter(iterable)
    while True:
        chunk = []
        for i in range(size):
            try:
                chunk.append(next(iterable))
            except StopIteration:
                if chunk:
                    yield tuple(chunk)
                return
        if chunk:
            yield tuple(chunk)

assert list(batch(range(1, 8), 3)) == [(1, 2, 3), (4, 5, 6), (7,)]      

In [13]:
def load_source(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            line = line.strip()
            if line:
                yield line
    return
                
lines = list(load_source('sample.txt'))
assert len(lines) == 16
assert lines[0] == '[1,1,3,1,1]'
assert lines[-1] == '[1,[2,[3,[4,[5,6,0]]]],8,9]'

In [14]:
def load_input(filename):
    source = batch(load_source(filename), 2)
    for left_expr, rigth_expr in source:
        yield (literal_eval(left_expr), literal_eval(rigth_expr))
    return

In [15]:
def solution_one(filename):
    counter = 0
    for line_num, pair in enumerate(load_input(filename), start=1):
        if compare(*pair):
            counter += line_num
    return counter
        

In [16]:
assert solution_one('sample.txt') == 13

In [17]:
sol = solution_one('input.txt')
print(f"Solution part one: {sol}")

Solution part one: 5806


## Part Two

Now, you just need to put all of the packets in the right order. Disregard the blank lines in your list of received packets.

The distress signal protocol also requires that you include two additional divider packets:

```
[[2]]
[[6]]
```

Using the same rules as before, organize all packets - the ones in your list of received packets as well as the two divider packets - into the correct order.

For the example above, the result of putting the packets in the correct order is:

```
[]
[[]]
[[[]]]
[1,1,3,1,1]
[1,1,5,1,1]
[[1],[2,3,4]]
[1,[2,[3,[4,[5,6,0]]]],8,9]
[1,[2,[3,[4,[5,6,7]]]],8,9]
[[1],4]
[[2]]
[3]
[[4,4],4,4]
[[4,4],4,4,4]
[[6]]
[7,7,7]
[7,7,7,7]
[[8,7,6]]
[9]
```

From the official documentation of Python:

> ### functools.cmp_to_key(func)

> Transform an old-style comparison function to a key function. Used with tools that accept key functions (such as sorted(), min(), max(), heapq.nlargest(), heapq.nsmallest(), itertools.groupby()). This function is primarily used as a transition tool for programs being converted from Python 2 which supported the use of comparison functions.

> A comparison function is any callable that accepts two arguments, compares them, and returns a negative number for less-than, zero for equality, or a positive number for greater-than. A key function is a callable that accepts one argument and returns another value to be used as the sort key.

In [18]:
from functools import cmp_to_key

def cmp_packets(l1, l2):
    result = compare(l1, l2)
    return -1 if result else 1
    

def solution_two(filename):
    lines = []
    for l1, l2 in load_input(filename):
        lines.append(l1)
        lines.append(l2)
    lines.append([[2]])
    lines.append([[6]])
    acc = 1
    for index, l in enumerate(sorted(lines, key=cmp_to_key(cmp_packets)), start=1):
        if l == [[2]] or l == [[6]]:
            acc *= index
    return acc

Afterward, locate the divider packets. To find the decoder key for this distress signal, you need to determine the indices of the two divider packets and multiply them together. (The first packet is at index 1, the second packet is at index 2, and so on.) In this example, the divider packets are 10th and 14th, and so the decoder key is 140.

In [19]:
assert solution_two('sample.txt') == 140

Organize all of the packets into the correct order. What is the decoder key for the distress signal?

In [20]:
sol = solution_two('input.txt')
print(f"Solution part two: {sol}")

Solution part two: 23600
